In [14]:
%%bash 
gcloud --version

Google Cloud SDK 398.0.0
bq 2.0.75
core 2022.08.12
gsutil 5.12


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update


In [ ]:
-- How many taxi trips were there on January 15?
SELECT count(*) from yellow_taxi_trips where tpep_pickup_datetime::date='2021-01-15';
-- 53024

In [1]:
-- Max tip for each day
SELECT tpep_pickup_datetime::date, max(tip_amount)
FROM public.yellow_taxi_trips
GROUP BY tpep_pickup_datetime::date
ORDER BY tpep_pickup_datetime::date

-- On which day it was the largest tip in January?
-- "2021-01-20" with 1140.44
SELECT tpep_pickup_datetime::date, tip_amount
FROM public.yellow_taxi_trips
where tpep_pickup_datetime::date>='2021-01-01' and tpep_pickup_datetime::date<='2021-01-31' and tip_amount = (
    SELECT max(tip_amount)
    FROM public.yellow_taxi_trips
    where tpep_pickup_datetime::date>='2021-01-01' and tpep_pickup_datetime::date<='2021-01-31'
)
 

SyntaxError: invalid syntax (1449867931.py, line 1)

In [ ]:
-- What was the most popular destination for passengers picked up in central park on January 14?
-- "Upper East Side South" with 97 trips 
SELECT z."Zone", n_trips from
(SELECT trips."DOLocationID",count(trips."DOLocationID") as n_trips
FROM public.yellow_taxi_trips as trips INNER JOIN zones as z on trips."PULocationID"=z."LocationID"
where tpep_pickup_datetime::date='2021-01-14' and tpep_pickup_datetime::date<='2021-01-31' and z."Zone"='Central Park'
group by trips."DOLocationID") as trips_DO INNER JOIN zones as z on trips_DO."DOLocationID"=z."LocationID"
ORDER BY n_trips DESC
LIMIT 1



In [ ]:
-- What's the pickup-dropoff pair with the largest average price for a ride (calculated based on total_amount)?
-- "Alphabet City / Unknown" with 2292.4 as the avg price

SELECT CONCAT(COALESCE(pu_zones."Zone", 'Unknown'),' / ',COALESCE(do_zones."Zone", 'Unknown')) as pickup_dropoff_pair, avg_price  from 
(
	SELECT trips."DOLocationID",trips."PULocationID", avg(trips.total_amount) as avg_price
	FROM public.yellow_taxi_trips as trips 
	group by trips."DOLocationID", trips."PULocationID"
	ORDER BY avg_price DESC
) as trips_avg_price 
LEFT OUTER JOIN zones as pu_zones on pu_zones."LocationID" = trips_avg_price."PULocationID" 
LEFT OUTER JOIN zones as do_zones on do_zones."LocationID" = trips_avg_price."DOLocationID"
LIMIT 1